Basic SoftMax multiclass regression with default (3, 3) parameter FV on only the MFCC features.

In [6]:
import os
import scipy.io as spio
import numpy as np
from os.path import join, isdir

DATA_DIR = '../data'
# 'file_name' 'class_name' 'class' 'eng' 'mfc' 'chroma' 't'
# 'keystrength' 'brightness' 'zerocross' 'roughness' 'inharmonic' 'hcdf'
#

In [24]:
# In-python MFC extraction (may not be necessary if we use MATLAB-only)
X = []
for basename in os.listdir(DATA_DIR)[:2]:
    path = join(DATA_DIR, basename)
    def loadmfc(base): return spio.loadmat(join(path, base))['DAT']['mfc'][0,0]
    mfc = np.concatenate([loadmfc(i).shape for i in os.listdir(path)])
    X.append([basename, mfc])
[classes, X] = zip(*X)
classes
# TODO: would be nice to have this fully in python.

('jazz', 'reggae')

In [ ]:
!/MATLAB_BINARY -nodisplay -nosplash -nodesktop -r "run('../tools/FV_concat.m');exit;"
mfcc = np.transpose(spio.loadmat(FVs)['FV'])
labels = spio.loadmat(LBs)['LB'][0]
N = mfcc.shape[1]
p = np.random.permutation(N)
mfcc, labels = mfcc[p], labels[p]
print('mfcc', mfcc.shape, 'labels', labels.shape)

trN = int(0.8 * N)
teN = N - trN
trX, trY = mfcc[:trN], labels[:trN]
teX, teY = mfcc[-teN:], labels[-teN:]

In [ ]:
# Random-restart softmax
from sklearn.linear_model import LogisticRegressionCV

reg = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs', max_iter=1000)
reg.fit(trX, trY)
reg.score(teX, teY)